In [13]:
# install in conda environment
# !conda install -c conda-forge pydub
# !conda install -c conda-forge ffmpeg

from os import path
from pydub import AudioSegment
import pandas as pd
import numpy as np
import pickle

In [6]:
from os import path
from pydub import AudioSegment

src = "Roar_of_the_Wild.mp3"
dst = "test.wav"

# convert wav to mp3                                                            
sound = AudioSegment.from_mp3(src)
sound.export(dst, format="wav")

<_io.BufferedRandom name='test.wav'>

In [17]:
import pandas as pd

df = pd.read_csv('../fma_metadata/tracks.csv', header=None)

# getting names for columns and index:
cnames = zip(df.iloc[0,1:], df.iloc[1,1:])
inames = list(df.iloc[0,:1])   

#drop the rows with column names (for columns and index)
df.drop([0,1],axis=0,inplace=True)
#set the indexes
df.set_index([0],inplace=True)
# set the names for columns and indexes
df.columns = pd.MultiIndex.from_tuples(cnames)
df.index.names = inames

df['set', 'subset'].head()
 
#df = df[(df['set', 'subset'] == 'small') | (df['set', 'subset'] == 'medium') | (df['set', 'subset'] == 'large')]

df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 106575 entries, track_id to 155320
Data columns (total 52 columns):
(album, comments)              106574 non-null object
(album, date_created)          103045 non-null object
(album, date_released)         70294 non-null object
(album, engineer)              15295 non-null object
(album, favorites)             106574 non-null object
(album, id)                    106574 non-null object
(album, information)           83149 non-null object
(album, listens)               106574 non-null object
(album, producer)              18060 non-null object
(album, tags)                  106574 non-null object
(album, title)                 105549 non-null object
(album, tracks)                106574 non-null object
(album, type)                  100066 non-null object
(artist, active_year_begin)    22711 non-null object
(artist, active_year_end)      5375 non-null object
(artist, associated_labels)    14271 non-null object
(artist, bio)                  

In [35]:
# adds an is_classical column
df['track', 'is_classical'] = False
count = 0
for i in range(1, len(df)):
    res = df['track', 'genres_all'].iloc[i].strip('][').split(', ')
    if ('5' in res):
        df['track', 'is_classical'].iloc[i] = True
        count += 1

In [1]:
pd.set_option('display.max_rows', 1000)
#classical = df[df['track', 'genre_top'] == 'Classical']
classical = df[df['track', 'is_classical'] == True]
print(len(classical))

info = pd.concat([classical['track', 'composer'], classical['track', 'title']], axis=1)
print("initial data length: ", len(info[info['track', 'composer'].notnull()]))

contain_composers = [r'.*Bach.*', r'.*Brahms.*', r'.*Mozart.*', r'.*Schubert.*', r'.*Beethoven.*', r'.*Paganini.*', r'.*Chopin.*', r'.*Grieg.*', r'.*Debussy.*', r'.*Mussorgsky.*', r'.*Liszt.*', r'.*Rachmaninoff.*', r'.*Schumann.*', r'.*Mendelssohn.*', r'.*Alkan.*', r'.*Vivaldi.*', r'.*Wagner.*', r'.*Satie.*', r'.*Camille.*', r'.*Pachelbel.*', r'.*Palestrina.*', r'.*Pizzetti.*', r'.*Bizet.*', r'.*Tchaikovsky.*', r'.*Dvorak.*', r'.*Handel.*', r'.*Bartok.*', r'.*Corelli.*', r'.*Albinoni.*', r'.*Ravel.*', r'.*Spohr.*']
composers = ['Bach', 'Brahms', 'Mozart', 'Schubert', 'Beethoven', 'Paganini', 'Chopin', 'Grieg', 'Debussy', 'Mussorgsky', 'Liszt', 'Rachmaninoff', 'Schumann', 'Mendelssohn', 'Alkan', 'Vivaldi', 'Wagner', 'Satie', 'Camille', 'Pachelbel', 'Palestrina', 'Pizzetti', 'Bizet', 'Tchaikovsky', 'Dvorak', 'Handel', 'Bartok', 'Corelli', 'Albinoni', 'Ravel', 'Spohr']

info['track', 'comp_from_title'] = info['track', 'title'].replace(contain_composers, composers, regex=True)
info['track', 'comp_from_title'].loc[~info['track', 'comp_from_title'].isin(composers)] = np.NaN
info['track', 'composer'] = info['track', 'composer'].fillna(info['track', 'comp_from_title'])

print("final data length: ", len(info[info['track', 'composer'].notnull()]))

print(info)
info.to_csv("track_info.csv")


NameError: name 'pd' is not defined

In [27]:
metadata_url = "https://homes.cs.washington.edu/~thickstn/media/musicnet_metadata.csv"

composer_to_period = {
    "Schubert": "Rom", 
    "Mozart": "Cla", 
    "Dvorak": "Rom",
    "Cambini": "Cla", 
    "Haydn": "Cla", 
    "Brahms": "Rom", 
    "Faure": "Rom", 
    "Ravel": "Mod", 
    "Bach": "Bar", 
    "Beethoven": "Cla"
    }

musicnet = pd.read_csv(metadata_url)
musicnet['period'] = musicnet['composer'].map(composer_to_period)

# create pkl file mapping file id to time period
file_path = 'musicnet_labels'
id_to_period = musicnet.set_index('id')['period'].to_dict()
labels_file = open(file_path,'wb')
pickle.dump(id_to_period, labels_file)
labels_file.close()

In [28]:
# read file
with open(file_path, 'rb') as handle:
    id2label = pickle.load(handle)